In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df=pd.read_csv('panl_result.csv')

In [3]:
def label(df):
    interval=30*1E6
    df['IsSpoof']=None
    for i in range(len(df)):
        if df.at[i,'order_type']=='EXECUTION' and df.at[i,'action']=='S' :
            time=df.at[i,'time']
            date=df.at[i,'date']
            tmp=df[(df['date']==date)&(df['time']<time)&(df['time']>(time-interval))]
            tmp=tmp[(tmp['order_type']=='CANCEL')&(tmp['action']=='B')]
            s=tmp['shares'].sum()
            if (s/df.at[i,'shares'])>4 or len(tmp)>=5:
                df.at[i,'IsSpoof']=True
            else:
                df.at[i,'IsSpoof']=False
                
        elif df.at[i,'order_type']=='EXECUTION' and df.at[i,'action']=='B' :
            time=df.at[i,'time']
            date=df.at[i,'date']
            tmp=df[(df['date']==date)&(df['time']<time)&(df['time']>(time-interval))]
            tmp=tmp[(tmp['order_type']=='CANCEL')&(tmp['action']=='S')]
            s=tmp['shares'].sum()
            if (s/df.at[i,'shares'])>4 or len(tmp)>=5:
                df.at[i,'IsSpoof']=True
            else:
                df.at[i,'IsSpoof']=False
            
        else:
            df.at[i,'IsSpoof']=None
        
    return df

In [4]:
df=label(df)

In [5]:
df.to_csv('panl_result_labeled.csv',index=False)